In [ ]:
import pygame              
import sys                 
import random              
import copy                


#  init window + constants (colors & fonts)

pygame.init()

WIN_WIDTH, WIN_HEIGHT = 800, 600
screen_surface = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))
pygame.display.set_caption("Mini‑Arcade")
pygame.display.set_icon(pygame.Surface((1, 1), pygame.SRCALPHA))  # tiny dummy icon

main_clock = pygame.time.Clock()

# colours 
WHITE  = (255, 255, 255)
BLACK  = (0, 0, 0)
RED    = (200, 0, 0)
GREEN  = (0, 200, 0)
LIGHT_GREEN = (120, 255, 120)
YELLOW = (255, 255, 0)
VIOLET = (140, 0, 255)
GREY   = (80, 80, 80)
PINK   = (255, 182, 193)
CREAM  = (255, 200, 100)
LIGHT_BLUE = (173, 216, 230)

# fonts 
MENU_FONT  = pygame.font.SysFont("arial", 48, bold=True)
TEXT_FONT  = pygame.font.SysFont("arial", 28)
SMALL_FONT = pygame.font.SysFont("arial", 22)


#  some helper functions 

def wait_for_key(valid_keys: tuple = (pygame.K_SPACE,)):
    """pause screen until user press one key from list"""
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit(); sys.exit()
            if event.type == pygame.KEYDOWN and event.key in valid_keys:
                return
        main_clock.tick(30)


def draw_text_lines(lines: list[str], top_y: int = 100, gap_px: int = 36, color=WHITE):
    screen_surface.fill(BLACK)
    for idx, line in enumerate(lines):
        txt_surface = TEXT_FONT.render(line, True, color)
        rect_here = txt_surface.get_rect(center=(WIN_WIDTH // 2, top_y + idx * gap_px))
        screen_surface.blit(txt_surface, rect_here)
    pygame.display.flip()


#   M A I N   M E N U


def main_menu() -> str:
    menu_items = ["Snake", "Sudoku", "Lane Racer", "Quit"]
    chosen_index = 0

    while True:
        # ---------- draw ----------
        screen_surface.fill(BLACK)
        hit_boxes: list[pygame.Rect] = []  # keep each text rect for mouse hover/click
        for idx, text in enumerate(menu_items):
            col = CREAM if idx == chosen_index else WHITE
            surf = MENU_FONT.render(text, True, col)
            r = surf.get_rect(center=(WIN_WIDTH // 2, 220 + idx * 80))
            screen_surface.blit(surf, r)
            hit_boxes.append(r)
        pygame.display.flip()

        # ---------- input ----------
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit(); sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key in (pygame.K_w, pygame.K_UP):
                    chosen_index = (chosen_index - 1) % len(menu_items)
                if event.key in (pygame.K_s, pygame.K_DOWN):
                    chosen_index = (chosen_index + 1) % len(menu_items)
                if event.key == pygame.K_SPACE:
                    return menu_items[chosen_index]
            if event.type == pygame.MOUSEMOTION:
                for idx, b in enumerate(hit_boxes):
                    if b.collidepoint(event.pos):
                        chosen_index = idx
                        break
            if event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                for idx, b in enumerate(hit_boxes):
                    if b.collidepoint(event.pos):
                        return menu_items[idx]
        main_clock.tick(60)


        

#  1.  S N A K E   
# ----------------

CELL_SIZE = 20
GRID_W, GRID_H = WIN_WIDTH // CELL_SIZE, WIN_HEIGHT // CELL_SIZE


def snake_game():

    def free_random_cell(the_snake: list[tuple[int, int]], obs: list[tuple[int, int]]):
        while True:
            cell = random.randrange(GRID_W), random.randrange(GRID_H)
            if cell not in the_snake and cell not in obs:
                return cell

    snake_body = [(GRID_W // 2, GRID_H // 2)]  # start centre
    dir_x, dir_y = 1, 0  # moving right first
    food_cell = free_random_cell(snake_body, [])
    obstacle_cells: list[tuple[int, int]] = []
    score_points = 0
    current_fps = 10

    draw_text_lines([
        "* W A S D to move.",
        "....Press SPACE to start...."
    ])
    wait_for_key()

    while True:
        # ---------- input ----------
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit(); sys.exit()
            if event.type == pygame.KEYDOWN:
                # i keep if chain, bit ugly but readable for me
                if event.key == pygame.K_w and dir_y != 1:
                    dir_x, dir_y = 0, -1
                if event.key == pygame.K_s and dir_y != -1:
                    dir_x, dir_y = 0, 1
                if event.key == pygame.K_a and dir_x != 1:
                    dir_x, dir_y = -1, 0
                if event.key == pygame.K_d and dir_x != -1:
                    dir_x, dir_y = 1, 0

        # ---------- update snake ----------
        new_head = ((snake_body[0][0] + dir_x) % GRID_W,
                    (snake_body[0][1] + dir_y) % GRID_H)

        if new_head in snake_body or new_head in obstacle_cells:
            break  # oops dead

        snake_body.insert(0, new_head)

        if new_head == food_cell:
            score_points += 10
            food_cell = free_random_cell(snake_body, obstacle_cells)
            # after 100 points start spawn obstacles (only one time) – small challenge
            if score_points >= 100 and not obstacle_cells:
                for _ in range(random.randint(5, 10)):
                    obstacle_cells.append(free_random_cell(snake_body, obstacle_cells + [food_cell]))
        else:
            snake_body.pop()  # move forward normally

        if score_points and score_points % 150 == 0:
            current_fps = 10 + 2 * (score_points // 150)  # game become faster slowly

        # ---------- draw ----------
        screen_surface.fill(BLACK)
        for x_cell, y_cell in snake_body:
            body_col = GREEN if (x_cell, y_cell) == snake_body[0] else LIGHT_GREEN
            pygame.draw.rect(screen_surface, body_col,
                             (x_cell * CELL_SIZE, y_cell * CELL_SIZE, CELL_SIZE, CELL_SIZE))
        for ox, oy in obstacle_cells:
            pygame.draw.rect(screen_surface, VIOLET, (ox * CELL_SIZE, oy * CELL_SIZE, CELL_SIZE, CELL_SIZE))
        pygame.draw.rect(screen_surface, RED, (food_cell[0] * CELL_SIZE, food_cell[1] * CELL_SIZE, CELL_SIZE, CELL_SIZE))

        score_surf = SMALL_FONT.render(f"Score: {score_points}", True, WHITE)
        screen_surface.blit(score_surf, (10, 10))

        pygame.display.flip()
        main_clock.tick(current_fps)

    # ---------- game over screen ----------
    draw_text_lines([f"GAME OVER – {score_points}", "R = replay   M = menu"], top_y=260)
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit(); sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_r:
                    return snake_game()  # restart
                if event.key == pygame.K_m:
                    return  # go back menu
        main_clock.tick(10)


#  2.  S U D O K U


SUDOKU_PIXELS = 540
CELL_PIX = SUDOKU_PIXELS // 9
BOARD_ORIGIN = ((WIN_WIDTH - SUDOKU_PIXELS) // 2, (WIN_HEIGHT - SUDOKU_PIXELS) // 2)


def generate_sudoku_board():
    base = list(range(9))
    row_order = [3 * (r % 3) + r // 3 for r in base]
    col_order = [3 * (c % 3) + c // 3 for c in base]
    pattern = lambda r, c: (3 * (r % 3) + r // 3 + c) % 9

    solved = [[pattern(r, c) + 1 for c in col_order] for r in row_order]

    # shuffle for randomness
    for band in range(3):
        random.shuffle(solved[band * 3:(band + 1) * 3])
    for stack in range(3):
        col_block = list(zip(*solved))[stack * 3:(stack + 1) * 3]
        random.shuffle(col_block)
        for r in range(9):
            row_copy = list(solved[r])
            row_copy[stack * 3:(stack + 1) * 3] = [col_block[c][r] for c in range(3)]
            solved[r] = row_copy

    puzzle_board = copy.deepcopy(solved)
    for hole in random.sample(range(81), 64):
        puzzle_board[hole // 9][hole % 9] = 0  # make holes

    return puzzle_board, solved


def sudoku_game():
    puzzle, answer = generate_sudoku_board()
    user_board = copy.deepcopy(puzzle)
    selected_cell: tuple[int, int] | None = None

    draw_text_lines([
        "* Click a cell, press 1 to 9",
        "* Backspace erases",
        "....Press SPACE to start....",
    ])
    wait_for_key()

    while True:
        # ---------- events ----------
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit(); sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                mouse_x, mouse_y = event.pos
                ox, oy = BOARD_ORIGIN
                if ox <= mouse_x < ox + SUDOKU_PIXELS and oy <= mouse_y < oy + SUDOKU_PIXELS:
                    selected_cell = ((mouse_x - ox) // CELL_PIX, (mouse_y - oy) // CELL_PIX)
            if event.type == pygame.KEYDOWN and selected_cell:
                cx, cy = selected_cell
                if event.key == pygame.K_BACKSPACE and puzzle[cy][cx] == 0:
                    user_board[cy][cx] = 0
                if event.unicode in "123456789" and puzzle[cy][cx] == 0:
                    user_board[cy][cx] = int(event.unicode)

        # ---------- draw board ----------
        screen_surface.fill(BLACK)
        ox, oy = BOARD_ORIGIN
        for row in range(9):
            for col in range(9):
                rect_cell = pygame.Rect(ox + col * CELL_PIX, oy + row * CELL_PIX, CELL_PIX, CELL_PIX)
                pygame.draw.rect(screen_surface, CREAM if selected_cell == (col, row) else WHITE, rect_cell, 1)
                num_here = user_board[row][col]
                if num_here:
                    num_color = LIGHT_BLUE if num_here == answer[row][col] else RED
                    num_surf = TEXT_FONT.render(str(num_here), True, num_color)
                    screen_surface.blit(num_surf, num_surf.get_rect(center=rect_cell.center))

        # thick 3x3 borders
        for i in range(10):
            line_width = 3 if i % 3 == 0 else 1
            pygame.draw.line(screen_surface, WHITE, (ox, oy + i * CELL_PIX), (ox + SUDOKU_PIXELS, oy + i * CELL_PIX), line_width)
            pygame.draw.line(screen_surface, WHITE, (ox + i * CELL_PIX, oy), (ox + i * CELL_PIX, oy + SUDOKU_PIXELS), line_width)

        pygame.display.flip()
        main_clock.tick(30)

        # check solved
        if all(user_board[r][c] == answer[r][c] for r in range(9) for c in range(9)):
            draw_text_lines(["S U C C E S S !", "R = replay   M = menu"], top_y=260)
            while True:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit(); sys.exit()
                    if event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_r:
                            return sudoku_game()
                        if event.key == pygame.K_m:
                            return
                main_clock.tick(10)

#  3.  L A N E   R A C E R


LANES_COUNT = 4
LANE_WIDTH = WIN_WIDTH // LANES_COUNT
CAR_WIDTH, CAR_HEIGHT = 60, 100

class Obstacle:
    def __init__(self, lane_index: int, kind: str):
        self.kind = kind
        self.speed_px = {"bike": 4, "car": 6, "truck": 8}[kind]
        self.color = {"bike": WHITE, "car": YELLOW, "truck": VIOLET}[kind]
        self.rect = pygame.Rect(
            lane_index * LANE_WIDTH + (LANE_WIDTH - CAR_WIDTH) // 2,
            -CAR_HEIGHT,
            CAR_WIDTH,
            CAR_HEIGHT,
        )

    def update(self) -> bool:
        self.rect.y += self.speed_px
        return self.rect.y > WIN_HEIGHT


def car_game():
    player_lane = LANES_COUNT // 2  # start middle
    player_rect = pygame.Rect(
        player_lane * LANE_WIDTH + (LANE_WIDTH - CAR_WIDTH) // 2,
        WIN_HEIGHT - CAR_HEIGHT - 20,
        CAR_WIDTH,
        CAR_HEIGHT,
    )

    obstacle_list: list[Obstacle] = []
    spawn_timer = 0
    score_frames = 0  # each frame is 1 point

    draw_text_lines([
        "* A/D or ←/→ to change lanes. Avoid everything!",
        "....Press SPACE to start...",
    ])
    wait_for_key()

    while True:
        # ---------- event ----------
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit(); sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key in (pygame.K_a, pygame.K_LEFT) and player_lane > 0:
                    player_lane -= 1
                if event.key in (pygame.K_d, pygame.K_RIGHT) and player_lane < LANES_COUNT - 1:
                    player_lane += 1
        player_rect.x = player_lane * LANE_WIDTH + (LANE_WIDTH - CAR_WIDTH) // 2

        # ---------- spawn obstacle ----------
        spawn_timer += 1
        if spawn_timer > 40:
            spawn_timer = 0
            kind_choice = random.choices(["bike", "car", "truck"], weights=[4, 3, 2])[0]
            obstacle_list.append(Obstacle(random.randrange(LANES_COUNT), kind_choice))

        # update obstacles 
        obstacle_list[:] = [ob for ob in obstacle_list if not ob.update()]

        # ---------- collision ----------
        if any(player_rect.colliderect(ob.rect) for ob in obstacle_list):
            break

        # ---------- draw ----------
        screen_surface.fill(GREY)
        # lane separators
        for i in range(1, LANES_COUNT):
            pygame.draw.line(screen_surface, WHITE, (i * LANE_WIDTH, 0), (i * LANE_WIDTH, WIN_HEIGHT), 3)
        # player
        pygame.draw.rect(screen_surface, PINK, player_rect)
        # obstacles
        for ob in obstacle_list:
            pygame.draw.rect(screen_surface, ob.color, ob.rect)

        # score (divide by 10 )
        score_frames += 1
        score_value = score_frames // 10
        score_surf = SMALL_FONT.render(f"Score {score_value}", True, BLACK)
        screen_surface.blit(score_surf, (10, 10))

        pygame.display.flip()
        main_clock.tick(60)

    # ---------- crash ----------
    draw_text_lines([
        f"C R A S H! Score {score_frames // 10}",
        "R = replay   M = menu",
    ], top_y=260)
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit(); sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_r:
                    return car_game()
                if event.key == pygame.K_m:
                    return
        main_clock.tick(10)

#   main program loop


if __name__ == "__main__":
    while True:
        picked = main_menu()
        if picked == "Snake":
            snake_game()
        elif picked == "Sudoku":
            sudoku_game()
        elif picked == "Lane Racer":
            car_game()
        else:
            pygame.quit(); sys.exit()
